In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torch.optim import lr_scheduler
from torch.utils.data import  DataLoader,datasets,Subset
from torchvision import datasets, models, transforms
from PIL import Image
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt 
import torchvision.models as models
import copy
from tqdm import tqdm
import os 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, f1_score


In [2]:
process_images = False
generate_datasets = False
generate_datasets_test = False

In [3]:
model = models.inception_v3(pretrained = True)
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [4]:
# sample execution (requires torchvision)
if process_images == True:

    for filename in tqdm(os.listdir("./task4_be9ai3nsdj/food/food/"), desc="Processing images..."):
    #     print(filename)
        if filename.endswith(".jpg"):

            input_image = Image.open("./task4_be9ai3nsdj/food/food/{}".format(filename))
            preprocess = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
            if torch.cuda.is_available():
                input_batch = input_batch.to('cuda')
                model.to('cuda')
            output = model(input_batch)
            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #         print(output[0])
            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            output_np = output.cpu().detach().numpy()
            np.save(
            "inception_model_processed/" + filename[:-4] + ".npy", output_np
            )

else:
    pass



In [5]:

indices = np.loadtxt("./task4_be9ai3nsdj/train_triplets.txt", dtype=str)
images_1 = [None] * indices.shape[0]
row = indices[0, :]
images_1[0] = np.load(f"inception_model_processed/{row[0]}.npy")
print(images_1[0])

[[-4.19511765e-01  7.17240453e-01 -4.17907596e-01 -7.96609223e-01
  -2.03577936e-01  1.47367656e-01 -9.54575181e-01  7.24247456e-01
   4.78866935e-01 -1.48642749e-01  6.16898164e-02 -9.36134160e-03
   7.30027318e-01 -3.93729389e-01  7.18339980e-01  3.16716641e-01
  -6.16172925e-02  1.09128654e+00  6.16158605e-01 -1.01752877e-02
  -1.98227137e-01 -7.58719027e-01 -4.01272267e-01 -7.05043554e-01
   8.74535143e-02 -1.61798730e-01 -8.48176479e-01 -9.72173512e-02
  -5.58912575e-01 -5.81693053e-02 -2.43935436e-01  6.41182721e-01
   5.84492683e-01  5.19972026e-01 -1.59786716e-01 -1.33104348e+00
   4.73614275e-01 -4.29091871e-01  4.83830273e-01 -1.50122297e+00
   4.86452997e-01 -4.18490380e-01 -4.06607121e-01  3.37400734e-01
  -1.13141209e-01 -4.31699783e-01 -2.56431490e-01 -1.19690806e-01
  -4.15895045e-01 -1.48666370e+00 -5.06790578e-01  8.69091630e-01
  -9.45580006e-01 -8.78918767e-01  1.76432669e-01  2.25945920e-01
   4.62989211e-02 -1.39700919e-01 -9.89195168e-01 -7.17386127e-01
  -1.11675

In [6]:
if generate_datasets == True:

    indices = np.loadtxt("./task4_be9ai3nsdj/train_triplets.txt", dtype=str)

    images_1 = [None] * indices.shape[0]
    images_2 = [None] * indices.shape[0]
    images_3 = [None] * indices.shape[0]

    for i in tqdm(range(indices.shape[0])):
        row = indices[i, :]
        images_1[i] = np.load(f"inception_model_processed/{row[0]}.npy")
        images_2[i] = np.load(f"inception_model_processed/{row[1]}.npy")
        images_3[i] = np.load(f"inception_model_processed/{row[2]}.npy")

    A = np.vstack(images_1)
    B = np.vstack(images_2)
    C = np.vstack(images_3)

    X1 = np.vstack((A, A, B))
    X2 = np.vstack((B, C, A))
    X3 = np.vstack((C, B, C))

    np.save("X1_InceptionV3.npy", X1)
    np.save("X2_InceptionV3.npy", X2)
    np.save("X3_InceptionV3.npy", X3)

X1 = np.load("X1_InceptionV3.npy").astype(np.float32)
X2 = np.load("X2_InceptionV3.npy").astype(np.float32)
X3 = np.load("X3_InceptionV3.npy").astype(np.float32)

dsize = X1.shape[0]
y_shape = (int(dsize / 3), 1)

Y = np.vstack((np.ones(y_shape), np.zeros(y_shape), np.ones(y_shape))).astype(np.float32)

In [7]:
if generate_datasets_test == True:

    indices = np.loadtxt("./task4_be9ai3nsdj/test_triplets.txt", dtype=str)

    images_1 = [None] * indices.shape[0]
    images_2 = [None] * indices.shape[0]
    images_3 = [None] * indices.shape[0]

    for i in tqdm(range(indices.shape[0])):
        row = indices[i, :]
        images_1[i] = np.load(f"inception_model_processed/{row[0]}.npy")
        images_2[i] = np.load(f"inception_model_processed/{row[1]}.npy")
        images_3[i] = np.load(f"inception_model_processed/{row[2]}.npy")

    X1_test = np.vstack(images_1)
    X2_test = np.vstack(images_2)
    X3_test = np.vstack(images_3)

    np.save("X1_test_InceptionV3.npy", X1_test)
    np.save("X2_test_InceptionV3.npy", X2_test)
    np.save("X3_test_InceptionV3.npy", X3_test)

X1_test = np.load("X1_test_InceptionV3.npy").astype(np.float32)
X2_test = np.load("X2_test_InceptionV3.npy").astype(np.float32)
X3_test = np.load("X3_test_InceptionV3.npy").astype(np.float32)

In [8]:
if X1 is not None:
    # If to ensure this cell isn't run twice.
    (
        X1_train,
        X1_validate,
        X2_train,
        X2_validate,
        X3_train,
        X3_validate,
        Y_train,
        Y_validate,
    ) = train_test_split(X1, X2, X3, Y, test_size=0.10, random_state=0, shuffle=True)

    # And freeing some memory
    del X1
    X1 = None
    del X2
    X2 = None
    del X3
    X3 = None


In [13]:
if not "scaler" in locals():
    scaler = StandardScaler()

    X1_train, X2_train, X3_train = np.vsplit(
        scaler.fit_transform(np.vstack((X1_train, X2_train, X3_train))), 3
    )

    # Transform the training data
    X1_validate = scaler.transform(X1_validate)
    X2_validate = scaler.transform(X2_validate)
    X3_validate = scaler.transform(X3_validate)

    # Transform the testing data
    X1_test = scaler.transform(X1_test)
    X2_test = scaler.transform(X2_test)
    X3_test = scaler.transform(X3_test)




In [118]:
class Densenet(torch.nn.Module):
    """
    Simple module implementing a feedforward neural network with
    num_layers layers of size width and input of size input_size.
    """

    def __init__(self, input_layer,out_features,seed = 10):
        super().__init__()
        self.hidden = nn.Linear(in_dim, 128) 
        self.output = nn.Linear(128, out_dim)
        # self.net.to(device)
        
    def forward(self, x):
        out = self.net(x)
        return out

In [127]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_function = nn.MSELoss()
outputs = Densenet(in_dim,out_dim)

In [140]:
Y_train.shape
train_dataset.shape
Y_train.size()

TypeError: 'int' object is not callable

In [133]:
# Run the training loop for defined number of epochs
in_dim =3000
out_dim = 1000
output_dim = 1
num_layers = 1
num_epochs = 5
out_features = np.zeros((4000,4000))

for epoch in range(0, num_epochs):

  # Print epoch
  print(f'Starting epoch {epoch+1}')

  # Set current loss value
  current_loss = 0.0

  # Iterate over the DataLoader for training data
  for i in enumerate(np.shape(train_dataset[0])):

    # Get inputs

    # Zero the gradients
    optimizer.zero_grad()

    # Perform forward pass
    outputs = Densenet(in_dim,out_dim)

    # Compute loss
    loss = loss_function(outputs, np.ravel(Y_train))

#     # Perform backward pass
    loss.backward()

    # Perform optimization
    optimizer.step()

    # Print statistics
    current_loss += loss.item()
    if i % 500 == 499:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 500))
        current_loss = 0.0

Starting epoch 1


TypeError: 'int' object is not callable

In [79]:
outputs

Densenet(
  (hidden): Linear(in_features=3000, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [18]:
test_triplets_id = pd.read_csv('task4_be9ai3nsdj/test_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()
triplets_id = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()
num_obsv = len(triplets_id)

train_dataset = np.array(([X1_train,X2_train,X3_train]))
train_dataset=train_dataset.reshape((160690,3*1000))
test_dataset = np.array(([X1_validate,X2_validate,X3_validate]))
test_dataset=test_dataset.reshape((17855,3*1000))

In [141]:
%%time
score_func = make_scorer(f1_score)
kf = KFold(n_splits=3, shuffle=False, random_state=None)
model = MLPClassifier(hidden_layer_sizes=5, activation='relu',
                      solver='adam', max_iter=10000, tol=1e-5, alpha=1e-3,early_stopping =True,
                      n_iter_no_change=100)
                      #warm_start=True)

train_dataset = np.array(([X1_train,X2_train,X3_train]))
train_dataset=train_dataset.reshape((160690,3*1000))

model.fit(train_dataset, np.ravel(Y_train))
test_dataset = np.array(([X1_validate,X2_validate,X3_validate]))
test_dataset=test_dataset.reshape((17855,3*1000))
prediction = model.predict(test_dataset)

print(f1_score(Y_validate, prediction))

A:\Anaconda\envs\torch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [ ]:
scores = cross_val_score(model, train_dataset, np.ravel(Y_train),scoring='accuracy', cv=5)

In [ ]:
print(scores)

In [ ]:
test_features=np.array(([X1_test,X2_test,X3_test]))
test_features=test_features.reshape(59544,3*1000)
prediction = model.predict(test_features)

In [ ]:
pd.DataFrame(prediction.T).to_csv('prediction.csv', index=False, header=None)